In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
import math
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
def select_key(num:int):
    no = num

    if no == 1:
        target_species = 'Coccinella novemnotata'
        key = 'novem'

    elif no == 2:
        target_species = 'Coccinella transversoguttata'
        key = 'trans'

    elif no == 3:
        target_species = 'Adalia bipunctata'
        key = 'bipun'

    elif no == 4:
        target_species = 'Hippodamia parenthesis'
        key = 'paren'

    return target_species,key

In [ ]:
num = 1
target_species,key= select_key(num)

In [ ]:
df = pd.read_csv("../Data/2nd_manipulated_data/20220808_" + str(key)+"_train_minmax.csv")

In [ ]:
df_target = df[df.Species=='Coccinella novemnotata'].iloc[:,8:].reset_index(drop=True)
df_absence = df[df.Species!='Coccinella novemnotata'].iloc[:,8:].reset_index(drop=True)

**귀무가설: 타겟종에 관한 부재지점과 존재지역의 종분포에는 차이가 없다.**

**대립가설:  타겟종에 관한 부재지점과 존재지역의 종분포에는 차이가 없다.**

In [ ]:
#t검정 

for j in tqdm(range(100)):
    random.seed(j)

    H_values_target = []
    for i in range(len(df_target)):
        target_row = df_target.iloc[[i]]
        # 전부 0으로 나온 행은 제외. 0으로 나누면 에러떠서 그냥 미리 뺌
        shannon_tmp = set(target_row.columns.tolist()) - set(target_row.columns[(target_row == 0).all()].tolist())
        shannon_tmp = list(shannon_tmp - {'Species'})

        H_value = 0
        for name in shannon_tmp:
            H_value = H_value + -1*float((target_row[name]/len(shannon_tmp))*math.log2((target_row[name]/len(shannon_tmp))))
        H_values_target.append(H_value)
    H_values_absence = []     
    for i in random.sample(df_absence.index.tolist(),len(df_target)):
        abs_row = df_absence.iloc[[i]]
        # 전부 0으로 나온 행은 제외. 0으로 나누면 에러떠서 그냥 미리 뺌
        shannon_tmp = set(abs_row.columns.tolist()) - set(abs_row.columns[(abs_row == 0).all()].tolist())
        shannon_tmp = list(shannon_tmp - {'Species'})

        H_value = 0
        for name in shannon_tmp:
            H_value = H_value + -1*float((abs_row[name]/len(shannon_tmp))*math.log2((abs_row[name]/len(shannon_tmp))))
        H_values_absence.append(H_value)      
    # pvalue가 0.05보다 큰경우만 출력
    if stats.ttest_ind(H_values_target, H_values_absence).pvalue >0.05:
        print(stats.ttest_ind(H_values_target, H_values_absence).pvalue )
